In [92]:
from itertools import product
import itertools
import numpy as np

In [93]:
def find_strongly_dominant_strategies(players, action_sets, payoff_matrices):
    def strictly_dominates(i, a, b):
        other_players = [j for j in players if j != i]
        other_action_profiles = product(*[action_sets[j] for j in other_players])
        for other_actions in other_action_profiles:
            profile_a = list(other_actions)
            profile_a.insert(i, a)
            profile_b = list(other_actions)
            profile_b.insert(i, b)
            if payoff_matrices[i][tuple(profile_a)] <= payoff_matrices[i][tuple(profile_b)]:
                return False
        return True

    dominant_strategies = {}
    for i in players:
        dominant_strategies[i] = []
        for a in action_sets[i]:
            if all(strictly_dominates(i, a, b) for b in action_sets[i] if b != a):
                dominant_strategies[i].append(a)

    return dominant_strategies

In [94]:
def find_weakly_dominant_strategies(players, action_sets, payoff_matrices):
    def weakly_dominates(i, a, b):
        strictly_better = False
        other_players = [j for j in players if j != i]
        for others in product(*[action_sets[j] for j in other_players]):
            profile_a = list(others)
            profile_b = list(others)
            profile_a.insert(i, a)
            profile_b.insert(i, b)
            ua = payoff_matrices[i][tuple(profile_a)]
            ub = payoff_matrices[i][tuple(profile_b)]
            if ua < ub:
                return False
            if ua > ub:
                strictly_better = True
        return strictly_better

    weakly_dominant_strategies = {}
    for i in players:
        weakly_dominant_strategies[i] = []
        for a in action_sets[i]:
            if all(weakly_dominates(i, a, b) for b in action_sets[i] if b != a):
                weakly_dominant_strategies[i].append(a)

    return weakly_dominant_strategies

In [95]:
def find_maxmin_strategies(players, action_sets, payoff_matrices):
    maxmin_values = {}
    maxmin_strategies = {}

    for i in players:
        min_payoffs = {}
        for a in action_sets[i]:
            worst_case = float('inf')
            other_players = [j for j in players if j != i]
            for others in product(*[action_sets[j] for j in other_players]):
                profile = list(others)
                profile.insert(i, a)
                payoff = payoff_matrices[i][tuple(profile)]
                worst_case = min(worst_case, payoff)
            min_payoffs[a] = worst_case
        maxmin_value = max(min_payoffs.values())
        maxmin_strats = [a for a, val in min_payoffs.items() if val == maxmin_value]
        maxmin_values[i] = maxmin_value
        maxmin_strategies[i] = maxmin_strats

    return maxmin_values, maxmin_strategies

In [96]:
# # example 1
# players = [0, 1, 2]

# action_sets = {
#     0: [0, 1, 2],
#     1: [0, 1, 2],
#     2: [0, 1, 2]
# }

# payoff_matrices = {
#     0: {
#         (0, 0, 0): 3, (0, 0, 1): 3, (0, 0, 2): 3,
#         (0, 1, 0): 3, (0, 1, 1): 3, (0, 1, 2): 3,
#         (0, 2, 0): 3, (0, 2, 1): 3, (0, 2, 2): 3,
#         (1, 0, 0): 1, (1, 0, 1): 1, (1, 0, 2): 1,
#         (1, 1, 0): 1, (1, 1, 1): 1, (1, 1, 2): 1,
#         (1, 2, 0): 1, (1, 2, 1): 1, (1, 2, 2): 1,
#         (2, 0, 0): 0, (2, 0, 1): 0, (2, 0, 2): 0,
#         (2, 1, 0): 0, (2, 1, 1): 0, (2, 1, 2): 0,
#         (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0,
#     },
#     1: {
#         (0, 0, 0): 2, (0, 0, 1): 2, (0, 0, 2): 2,
#         (0, 1, 0): 1, (0, 1, 1): 1, (0, 1, 2): 1,
#         (0, 2, 0): 0, (0, 2, 1): 0, (0, 2, 2): 0,
#         (1, 0, 0): 2, (1, 0, 1): 2, (1, 0, 2): 2,
#         (1, 1, 0): 1, (1, 1, 1): 1, (1, 1, 2): 1,
#         (1, 2, 0): 0, (1, 2, 1): 0, (1, 2, 2): 0,
#         (2, 0, 0): 2, (2, 0, 1): 2, (2, 0, 2): 2,
#         (2, 1, 0): 1, (2, 1, 1): 1, (2, 1, 2): 1,
#         (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0,
#     },
#     2: {
#         (0, 0, 0): 1, (0, 0, 1): 0, (0, 0, 2): 2,
#         (0, 1, 0): 1, (0, 1, 1): 0, (0, 1, 2): 2,
#         (0, 2, 0): 1, (0, 2, 1): 0, (0, 2, 2): 2,
#         (1, 0, 0): 1, (1, 0, 1): 0, (1, 0, 2): 2,
#         (1, 1, 0): 1, (1, 1, 1): 0, (1, 1, 2): 2,
#         (1, 2, 0): 1, (1, 2, 1): 0, (1, 2, 2): 2,
#         (2, 0, 0): 1, (2, 0, 1): 0, (2, 0, 2): 2,
#         (2, 1, 0): 1, (2, 1, 1): 0, (2, 1, 2): 2,
#         (2, 2, 0): 1, (2, 2, 1): 0, (2, 2, 2): 2,
#     }
# }

In [104]:
#example 6.2 from book
players = [0, 1]

action_sets = {
    0: ['NC', 'C'],
    1: ['NC', 'C']
}

payoff_matrices = {
    0: {
        ('NC', 'NC'): -2,
        ('NC', 'C'): -10,
        ('C', 'NC'): -1,
        ('C', 'C'): -5,
    },
    1: {
        ('NC', 'NC'): -2,
        ('NC', 'C'): -1,
        ('C', 'NC'): -10,
        ('C', 'C'): -5,
    }
}

In [98]:
# # example 6.6 from book
# players = [0, 1]

# action_sets = {
#     0: ['A', 'B'],  # Player 1's actions
#     1: ['A', 'B']   # Player 2's actions
# }

# payoff_matrices = {
#     0: {  # Player 1's payoffs
#         ('A', 'A'): -1,
#         ('A', 'B'): -0.5,
#         ('B', 'A'): -1,
#         ('B', 'B'): -1
#     },
#     1: {  # Player 2's payoffs
#         ('A', 'A'): -1,
#         ('A', 'B'): -1,
#         ('B', 'A'): -0.5,
#         ('B', 'B'): -1
#     }
# }


In [99]:
# #example 6.9 from book
# players = [0, 1]

# action_sets = {
#     0: ['Rock', 'Paper', 'Scissors'],
#     1: ['Rock', 'Paper', 'Scissors']
# }

# payoff_matrices = {
#     0: {  # Player 1's payoffs
#         ('Rock', 'Rock'): 0,     ('Rock', 'Paper'): -1,  ('Rock', 'Scissors'): 1,
#         ('Paper', 'Rock'): 1,    ('Paper', 'Paper'): 0,  ('Paper', 'Scissors'): -1,
#         ('Scissors', 'Rock'): -1,('Scissors', 'Paper'): 1, ('Scissors', 'Scissors'): 0
#     },
#     1: {  # Player 2's payoffs
#         ('Rock', 'Rock'): 0,     ('Rock', 'Paper'): 1,   ('Rock', 'Scissors'): -1,
#         ('Paper', 'Rock'): -1,   ('Paper', 'Paper'): 0,  ('Paper', 'Scissors'): 1,
#         ('Scissors', 'Rock'): 1, ('Scissors', 'Paper'): -1, ('Scissors', 'Scissors'): 0
#     }
# }


In [100]:
# #video example

# players = [0, 1, 2]  # 0: Gus, 1: Yelnic, 2: Tolbert

# action_sets = {
#     0: ['Goes to bar', 'Stays home'],  # Gus
#     1: ['go to bar', 'stays home'],    # Yelnic
#     2: ['Goes to bar', 'Stays home']   # Tolbert
# }

# payoff_matrices = {
#     0: {  # Gus
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 2,
#         ('Goes to bar', 'go to bar', 'Stays home'): 2,
#         ('Goes to bar', 'stays home', 'Stays home'): 0,
#         ('Stays home', 'go to bar', 'Goes to bar'): 1,
#         ('Stays home', 'stays home', 'Goes to bar'): 1,
#         ('Stays home', 'go to bar', 'Stays home'): 1,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     },
#     1: {  # Yelnic
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 1,
#         ('Goes to bar', 'go to bar', 'Stays home'): 2,
#         ('Goes to bar', 'stays home', 'Stays home'): 1,
#         ('Stays home', 'go to bar', 'Goes to bar'): 2,
#         ('Stays home', 'stays home', 'Goes to bar'): 1,
#         ('Stays home', 'go to bar', 'Stays home'): 0,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     },
#     2: {  # Tolbert
#         ('Goes to bar', 'go to bar', 'Goes to bar'): -1,
#         ('Goes to bar', 'stays home', 'Goes to bar'): 2,
#         ('Goes to bar', 'go to bar', 'Stays home'): 1,
#         ('Goes to bar', 'stays home', 'Stays home'): 1,
#         ('Stays home', 'go to bar', 'Goes to bar'): 2,
#         ('Stays home', 'stays home', 'Goes to bar'): 0,
#         ('Stays home', 'go to bar', 'Stays home'): 1,
#         ('Stays home', 'stays home', 'Stays home'): 1
#     }
# }

In [101]:
def get_input():
    num_players = int(input("Enter number of players: "))
    players = [i for i in range(num_players)]
    action_sets = {}
    for p in range(num_players):
        actions = input(f"Enter space-separated action for player {p} : ").split()
        action_sets[p] = [a for a in actions]

    all_profiles = list(product(*[action_sets[i] for i in range(num_players)]))
    payoff_matrices = {p: {} for p in range(num_players)}

    print("The index represents players i choice eg (C, NC) is player 0, player 1")
    for player in range(num_players):
        print(f"For Player {player}")
        for profile in all_profiles:
            input_utility = int(input(f"Profile {profile}: "))
            # print(input_utility)
            payoff_matrices[player][profile] = input_utility

    return players, action_sets, payoff_matrices

# players, action_sets, payoff_matrices = get_input()
# print(players, action_sets, payoff_matrices)

In [105]:
#part a
dominant_strategies = find_strongly_dominant_strategies(players, action_sets, payoff_matrices)
print("Strongly Dominant Strategies:")
for i in players:
    print(f"Player {i}: {dominant_strategies[i]}")

print()

# part b
weakly_dominant_strategies_dict = find_weakly_dominant_strategies(players, action_sets, payoff_matrices)
print("Weakly Dominant Strategies:")
for i in players:
    print(f"Player {i}: {weakly_dominant_strategies_dict[i]}")
print()

# part c
maxmin_values, maxmin_strategies = find_maxmin_strategies(players, action_sets, payoff_matrices)
print("Maxmin Values and Strategies:")
for i in players:
    print(f"Player {i}: Maxmin Value = {maxmin_values[i]}, Maxmin Strategies = {maxmin_strategies[i]}")
print()

#part d
if all(dominant_strategies[i] for i in players):
    equilibria = list(product(*[dominant_strategies[i] for i in players]))
    print("Strongly Dominant Strategy Equilibria:")
    for eq in equilibria:
        print(eq)
else:
    print("No Strongly Dominant Strategy Equilibrium exists.")
print()

#part e
if all(weakly_dominant_strategies_dict[i] for i in players):
    # Generate all combinations of weakly dominant strategies
    equilibria = list(product(*[weakly_dominant_strategies_dict[i] for i in players]))
    print("Weakly Dominant Strategy Equilibria:")
    for eq in equilibria:
        print(eq)
else:
    print("No Weakly Dominant Strategy Equilibrium exists.")

Strongly Dominant Strategies:
Player 0: ['C']
Player 1: ['C']

Weakly Dominant Strategies:
Player 0: ['C']
Player 1: ['C']

Maxmin Values and Strategies:
Player 0: Maxmin Value = -5, Maxmin Strategies = ['C']
Player 1: Maxmin Value = -5, Maxmin Strategies = ['C']

Strongly Dominant Strategy Equilibria:
('C', 'C')

Weakly Dominant Strategy Equilibria:
('C', 'C')
